# Estimate upper bound of uniform dist
$\mathcal{U}(0, \theta)$

In [17]:
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt

theta = 5
nsamp = 10
nsim = 10000

def mse(est):
    return (est - theta)**2

def naive_est(x):
    return 2*np.mean(x)

def mle_ub_est(x):
    ssize = x.shape[0]
    return (ssize + 1)/ssize * np.max(x)

def mmse_est(x):
    ssize = x.shape[0]
    return (ssize + 2)/(ssize + 1) * np.max(x)

def stratify_uniform_slope(y, intercept = True):
    nsize = y.shape[0]
    x = (np.arange(nsize) + 1) / (nsize + 1)
    x_aligned = np.sort(x)
    y_aligned = np.sort(y)
    
    endog = y_aligned
    if intercept:
        exog = np.column_stack([np.ones_like(x_aligned), x_aligned])
        param_hat = np.linalg.inv(exog.T @ exog) @ exog.T @ endog
    else:
        exog = x_aligned
        param_hat = np.array([1/np.sum(exog**2) * np.sum(exog * endog)])
    
    return param_hat[-1]

def strat_biased(x):
    return stratify_uniform_slope(x, intercept=False)

def strat_unbiased(x):
    return stratify_uniform_slope(x, intercept=True)


estimators = [naive_est, mle_ub_est, mmse_est, strat_biased, strat_unbiased]

est_mse = np.empty((nsim, len(estimators)))
for i in range(nsim):
    x = np.random.uniform(0, theta, nsamp)
    est_mse[i,:] = np.array([mse(est(x)) for est in estimators])
    
    
est = np.mean(est_mse, axis=0)    
est_names = [e.__name__ for e in estimators]
print('Mean Squared Error')
print(', '.join(est_names))
print(np.around(est, 3))


Mean Squared Error
naive_est, mle_ub_est, mmse_est, strat_biased, strat_unbiased
[0.828 0.207 0.205 0.545 0.73 ]
